In [98]:
import random, numpy as np, pandas as pd, torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import yfinance as yf, ta
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

SEED = 42
random.seed(SEED);  np.random.seed(SEED)
torch.manual_seed(SEED);  torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic, torch.backends.cudnn.benchmark = True, False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [99]:
start_date = "2019-01-01"
end_date   = datetime.today().strftime('%Y-%m-%d')
#end_date = "2025-05-05"
df = yf.download("AAPL", start=start_date, end=end_date)

# flatten MultiIndex cols if present
df.columns = [c[0] if isinstance(c, tuple) else c for c in df.columns]
df = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy().dropna()

# engineered + TA features
df['Return']       = df['Close'].pct_change()
df['Candle_Body']  = df['Close'] - df['Open']
df['Range']        = df['High']  - df['Low']
df['rsi']          = ta.momentum.RSIIndicator(df['Close']).rsi()
df['macd']         = ta.trend.MACD(df['Close']).macd_diff()
df['ema_10']       = ta.trend.EMAIndicator(df['Close'], 10).ema_indicator()
df['bb_bbw']       = ta.volatility.BollingerBands(df['Close']).bollinger_wband()
df['adx']          = ta.trend.ADXIndicator(df['High'], df['Low'], df['Close']).adx()
df['stoch']        = ta.momentum.StochasticOscillator(df['High'], df['Low'], df['Close']).stoch()

df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)
df.dropna(inplace=True)
print("Final DataFrame shape:", df.shape)


/var/folders/l6/kt268nls2sqcx51m4w_z09pc0000gn/T/ipykernel_71433/1144416275.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("AAPL", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Final DataFrame shape: (1635, 14)



/var/folders/l6/kt268nls2sqcx51m4w_z09pc0000gn/T/ipykernel_71433/1144416275.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/var/folders/l6/kt268nls2sqcx51m4w_z09pc0000gn/T/ipykernel_71433/1144416275.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [100]:
feature_cols = df.columns.tolist()
feature_cols.remove('Close')        # keep Close only for label
scaler       = MinMaxScaler()
scaled_feats = scaler.fit_transform(df[feature_cols])
scaled_close = MinMaxScaler().fit_transform(df[['Close']])
scaled_all   = np.concatenate([scaled_close, scaled_feats], axis=1)  # shape (*,14)

SEQ_LEN  = 30
X, y     = [], []
thresh   = 0.003

for i in range(SEQ_LEN, len(scaled_all) - 3):
    window = scaled_all[i-SEQ_LEN:i]
    change = scaled_all[i+3][0] - scaled_all[i][0]  # 3-day look-ahead on Close
    if   change >  thresh:  y.append(1)
    elif change < -thresh:  y.append(0)
    else:                   continue
    X.append(window)

X, y = np.array(X), np.array(y)
print(f"Dataset: {X.shape[0]} samples, window {X.shape[1:]} (seq, features)")


Dataset: 1333 samples, window (30, 14) (seq, features)


In [101]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
ds       = TensorDataset(X_tensor, y_tensor)

train_len = int(0.8 * len(ds))
val_len   = int(0.1 * len(ds))
test_len  = len(ds) - train_len - val_len
train_ds, val_ds, test_ds = random_split(ds, [train_len, val_len, test_len],
                                         generator=torch.Generator().manual_seed(SEED))

BATCH = 64
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH)
test_loader  = DataLoader(test_ds,  batch_size=BATCH)

print("Loader check:", next(iter(train_loader))[0].shape)  # (B,30,14)

# Save test data for external evaluation (e.g., FinBERT integration)
X_test_np = X_tensor[test_ds.indices].numpy()
y_test_np = y_tensor[test_ds.indices].numpy()
#np.save("X_test_pre_may.npy", X_test_np)
#np.save("y_test_pre_may.npy", y_test_np)


Loader check: torch.Size([64, 30, 14])


In [102]:
class LSTM_CNN_Attention(nn.Module):
    def __init__(self, input_size, lstm_hidden=64, cnn_out=24, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, lstm_hidden, batch_first=True, bidirectional=True)
        self.ln   = nn.LayerNorm(2 * lstm_hidden)

        self.conv1 = nn.Conv1d(2*lstm_hidden, cnn_out, 3, padding=1)
        self.bn1   = nn.BatchNorm1d(cnn_out)
        self.conv2 = nn.Conv1d(cnn_out, cnn_out, 3, padding=1)
        self.bn2   = nn.BatchNorm1d(cnn_out)
        self.cnn_drop = nn.Dropout(0.2)

        self.attn_fc  = nn.Linear(cnn_out, 32)
        self.attn_vec = nn.Linear(32, 1)

        self.dropout = nn.Dropout(0.3)
        self.head    = nn.Sequential(
            nn.Linear(cnn_out, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        h, _  = self.lstm(x)
        h     = self.ln(h)
        c     = h.permute(0,2,1)
        c     = F.relu(self.bn1(self.conv1(c)))
        c     = F.relu(self.bn2(self.conv2(c)))
        c     = self.cnn_drop(c).permute(0,2,1)

        e  = torch.tanh(self.attn_fc(c))
        w  = F.softmax(self.attn_vec(e), dim=1)
        ctx = torch.sum(w * c, dim=1)
        out = self.dropout(ctx)
        return self.head(out)


In [103]:
model      = LSTM_CNN_Attention(input_size=X_tensor.shape[2]).to(device)
optimizer  = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion  = nn.CrossEntropyLoss()

# ↓ scheduler cuts LR by ½ if val-acc hasn’t improved for 2 epochs
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode="max", factor=0.5, patience=2)   # ← no verbose


best_state, best_val, patience, wait = None, 0, 5, 0
EPOCHS = 50

for epoch in range(1, EPOCHS + 1):
    # ----- train -----
    model.train()
    tr_loss = tr_corr = tr_cnt = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss   = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        tr_loss += loss.item() * yb.size(0)
        tr_corr += (logits.argmax(1) == yb).sum().item()
        tr_cnt  += yb.size(0)
    tr_acc = tr_corr / tr_cnt

    # ----- validate -----
    model.eval()
    v_corr = v_cnt = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds  = model(xb).argmax(1)
            v_corr += (preds == yb).sum().item()
            v_cnt  += yb.size(0)
    v_acc = v_corr / v_cnt

    # Step scheduler on validation accuracy
    scheduler.step(v_acc)

    current_lr = optimizer.param_groups[0]["lr"]
    print(f"Epoch {epoch:02}  TrainAcc {tr_acc:.4f}  ValAcc {v_acc:.4f}  LR {current_lr:.5f}")

    # ----- early stop -----
    if v_acc > best_val:
        best_val, best_state, wait = v_acc, {k: v.cpu() for k, v in model.state_dict().items()}, 0
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping")
            break

# Restore best-validation weights
model.load_state_dict(best_state)
model.to(device)
# Save the best model
torch.save(model.state_dict(), "pre-may-model.pth")
# Save test data





Epoch 01  TrainAcc 0.4972  ValAcc 0.5038  LR 0.00100
Epoch 02  TrainAcc 0.5779  ValAcc 0.5714  LR 0.00100
Epoch 03  TrainAcc 0.5732  ValAcc 0.4135  LR 0.00100
Epoch 04  TrainAcc 0.5797  ValAcc 0.4511  LR 0.00100
Epoch 05  TrainAcc 0.5732  ValAcc 0.5564  LR 0.00050
Epoch 06  TrainAcc 0.5929  ValAcc 0.5714  LR 0.00050
Epoch 07  TrainAcc 0.5863  ValAcc 0.5789  LR 0.00050
Epoch 08  TrainAcc 0.5882  ValAcc 0.5940  LR 0.00050
Epoch 09  TrainAcc 0.5947  ValAcc 0.5489  LR 0.00050
Epoch 10  TrainAcc 0.6126  ValAcc 0.5639  LR 0.00050
Epoch 11  TrainAcc 0.6013  ValAcc 0.5789  LR 0.00025
Epoch 12  TrainAcc 0.6107  ValAcc 0.5865  LR 0.00025
Epoch 13  TrainAcc 0.5994  ValAcc 0.6015  LR 0.00025
Epoch 14  TrainAcc 0.6126  ValAcc 0.5940  LR 0.00025
Epoch 15  TrainAcc 0.6004  ValAcc 0.6015  LR 0.00025
Epoch 16  TrainAcc 0.6088  ValAcc 0.5789  LR 0.00013
Epoch 17  TrainAcc 0.6107  ValAcc 0.5714  LR 0.00013
Epoch 18  TrainAcc 0.6173  ValAcc 0.5639  LR 0.00013
Early stopping


In [104]:
model.eval()
t_corr = t_cnt = 0
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        t_corr += (model(xb).argmax(1)==yb).sum().item(); t_cnt += yb.size(0)
print("Test Accuracy:", round(t_corr/t_cnt, 4))



Test Accuracy: 0.6493
